# AlphaFold metrics
Created by Andreas 04.04.2025

This is the main notebook to add the following informations and metrics to a given AlphaFold run:

1. [RMSD](#3-rmsd-with-pymol)
2. [DockQ](#4-dockq)
3. [ipSAE](#5-ipsae-metric)
4. [Interface Interaction metrics](#6-interaction-metrics)

The sections (e.g. RMSD, DockQ) can be run individually to recalculate only some metrics. For this, set the *load_previous* setting to True.

The benchmark set of Lee et al. encodes information about a PPI in a string (e.g. *PF10208_PF00012_6H9U_B_resi130_resi169.A_resi30_resi406*). This script breaks it down into the PDB ID, PFAM ID, and ELM instance. Also, as it was worked with minimal interacting regions (i.e. only the motif or domain), the included chains (i.e. their IDs) as well as the boundaries were either extracted from the prediction name (DDI) or determined by comparing with the experimentally solved structure.

As input, the .pdb files need to be stored in the corrosponding benchmark set:

```bash
├── DDI
│   ├── known_DDI
│   └── random_DDI
├── DMI
│   ├── known_DMI
│   ├── mutations_DMI
│   └── random_DMI
```

Every structure inside this folder should follow this layout:
```bash
├── DEG_APCC_KENBOX_2_4GGD
│   ├── ranked_0.pdb
│   ├── ranked_1.pdb
│   ├── ranked_2.pdb
│   ├── ranked_3.pdb
│   └── ranked_4.pdb
```

The experimentally solved structures should have the following folder structure:

```bash
├── DDI
├── DDI_hydrogens
├── DMI
├── DMI_hydrogens
```

with the hydrogen folders containing the structures with added hydrogen.

It is also necessary to provide the output file of the AlphaFold run. For AlphaFold 2, this file was created by Lee et al., while for AlphaFold 3 the <a href="AF3 raw output parsing.ipynb">AF3 raw output parsing.ipynb</a> notebook is able to generate one. This file should list all AlphaFold predictions with AlphaFold derived metrics like the model cofidence, the number of clashes etc.

By default, you specify a *resource* path with a *AF2*/*AF3* and *solved* folder inside in it. Also, a *AF3_hydrogens* folder can be included.

### 0 Imports and Settings

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.axes._axes import Axes
from matplotlib.figure import Figure
from pathlib import Path
from sklearn.metrics import roc_curve, roc_auc_score
import re
import tempfile
import shutil
import os
import subprocess
import sys
stdout, stderr = sys.stdout, sys.stderr
from typing import Literal

import pymol
from Bio.SeqUtils import seq1
from Bio.PDB import PDBParser
from Bio.PDB.Structure import Structure as BioPy_PDBStructure
from Bio.PDB.Model import Model as BioPy_PDBModel
from Bio.PDB.Chain import Chain
from Bio.PDB.PDBExceptions import PDBConstructionException
parser = PDBParser(QUIET=True)

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [2]:
# Settings

# Which AF output should be parsed
af_mode: Literal["AF2", "AF3"] = "AF2"
# Should be the hydrogens folder used?
af3_hydrogens: bool = True

# Path to resource folder with the structures and metadata tables
path_resources = Path("/Users/imb/Desktop")
# Path to the Luck Drive folder (used for ipSAE metric to get the json file)
path_AF_luck_drive = Path("/Volumes/imb-luckgr/imb-luckgr2/projects/AlphaFold")
if af_mode == "AF3":
    path_AF_luck_drive = path_AF_luck_drive / "AlphaFold3"

# Paths to the local folders
path_AF = path_resources / ((af_mode + "_hydrogens") if (af3_hydrogens and af_mode == "AF3") else af_mode)
path_solved = path_resources / "solved"

# The path to the ipsae.py
path_ipsae_script = Path("../external/ipsae.py")

# If set to true, load the previous dataframe
load_previous = False

# To parse structures, Pymol should be run headless. However, for debugging the code the GUI may be helpful
pymol_headless = True

In [3]:
if not pymol_headless:
    pymol.finish_launching()
    sys.stdout = stdout # Needed in case of debugging to redicrct 
    sys.stderr = stderr

In [4]:
def enhance_dataframe():
    """ Reorder the columns in the dataAF dataframe and convert to an appropriate dtype """
    global dataAF
    # Some columns are integer, but contain None values. Default behaviour of pandas is to use dtype float. Therefore change the dtype to the pd.Int64Dtype allowing None
    for c in ["chainA_start", "chainA_end", "chainB_start", "chainB_end", "num_mutations", "num_align_atoms_domain", "num_align_resi_domain", "hbonds", "salt_bridges", "hydrophobic_interactions", "disulfide_bonds"]:
        if c not in dataAF.columns:
            print(f"Column {bcolors.FAIL}{c}{bcolors.ENDC} not (yet) in data frame")
            continue
        dataAF[c] = dataAF[c].astype(pd.Int64Dtype())

    def _reorder_column(c:list[str], column: str, prev_column: str = None, index:int = None):
        if column not in c:
            print(f"Column {bcolors.FAIL}{column}{bcolors.ENDC} not (yet) in data frame")
            return
        if index is None:
            if prev_column not in c:
                print(f"Column {bcolors.FAIL}{prev_column}{bcolors.ENDC} (used for sorting) not (yet) in data frame")
                return
            index = c.index(prev_column) + 1
        c.remove(column)
        c.insert(index, column)

    # Reordering of the columns
    c = list(dataAF.columns)
    if af_mode == "AF2":
        _reorder_column(c, "run_id", index=1)
        _reorder_column(c, "benchmark_set", index=2)
    elif af_mode == "AF3":
        _reorder_column(c, "benchmark_set", index=1)
    _reorder_column(c, "prediction_name", prev_column="benchmark_set")
    _reorder_column(c, "model_id", prev_column="prediction_name")
    if af_mode == "AF3": 
        _reorder_column(c, "ranking_score", prev_column="model_id")
        _reorder_column(c, "chainA_length", prev_column="ranking_score")
    else:
        _reorder_column(c, "chainA_length", prev_column="model_id")
    _reorder_column(c, "chainB_length", prev_column="chainA_length")
    _reorder_column(c, "chainA_id", prev_column="chainB_length")
    _reorder_column(c, "chainB_id", prev_column="chainA_id")
    _reorder_column(c, "chainA_start", prev_column="chainB_id")
    _reorder_column(c, "chainA_end", prev_column="chainA_start")
    _reorder_column(c, "chainB_start", prev_column="chainA_end")
    _reorder_column(c, "chainB_end", prev_column="chainB_start")
    _reorder_column(c, "PDB_id", prev_column="chainB_end")
    _reorder_column(c, "ELM_instance", prev_column="PDB_id")
    _reorder_column(c, "DDI_pfam_id", prev_column="ELM_instance")
    _reorder_column(c, "PDB_id_random_paired", prev_column="DDI_pfam_id")
    _reorder_column(c, "ELM_instance_random_paired", prev_column="PDB_id_random_paired")
    _reorder_column(c, "DDI_pfam_id_random_paired", prev_column="ELM_instance_random_paired")
    _reorder_column(c, "sequence_initial", prev_column="DDI_pfam_id_random_paired")
    _reorder_column(c, "sequence_mutated", prev_column="sequence_initial")
    _reorder_column(c, "num_mutations", prev_column="sequence_mutated")

    _reorder_column(c, "align_score_domain", prev_column="num_atom_atom_contact")
    _reorder_column(c, "num_align_atoms_domain", prev_column="align_score_domain")
    _reorder_column(c, "num_align_resi_domain", prev_column="num_align_atoms_domain")
    _reorder_column(c, "RMSD_domain", prev_column="num_align_resi_domain")
    _reorder_column(c, "RMSD_backbone_peptide", prev_column="RMSD_domain")
    _reorder_column(c, "RMSD_all_atom_peptide", prev_column="RMSD_backbone_peptide")
    _reorder_column(c, "RMSD_all_atom", prev_column="RMSD_all_atom_peptide")

    _reorder_column(c, "buried_area", prev_column="Fnonnat")
    _reorder_column(c, "min_distance", prev_column="buried_area")
    _reorder_column(c, "disulfide_bonds", prev_column="min_distance")
    _reorder_column(c, "salt_bridges", prev_column="disulfide_bonds")
    _reorder_column(c, "hbonds", prev_column="salt_bridges")
    _reorder_column(c, "hydrophobic_interactions", prev_column="hbonds")
    

    dataAF = dataAF[c]


In [5]:
# Load data
if load_previous:
    dataAF = pd.read_csv(path_AF / (path_AF.name + "_metrics.tsv"), sep="\t")
else:
    # Read in the AF data
    if af_mode == "AF2":
        dataAF = pd.read_excel(path_AF / "AF2_extension_metrics.xlsx")

        print(dataAF.columns.tolist())

        # Drop columns to recalculate them
        dataAF.drop(columns=["label"], inplace=True)


        # Adding benchmark set column
        benchmark_set_replace_dict = {"known extension": "known_extension", "random extension" : "random_extension"}
        dataAF["benchmark_set"] = None
        dataAF["num_mutations"] = None

        for i, row in dataAF.iterrows():
            if row["mutation_in_motif"] == "1":
                dataAF.at[i, "num_mutations"] = 1
            elif row["mutation_in_motif"] == "2":
                dataAF.at[i, "num_mutations"] = 2
            benchmark_set = benchmark_set_replace_dict[row["mutation_in_motif"]]
            dataAF.at[i, "benchmark_set"] = benchmark_set
        dataAF.drop(columns=["mutation_in_motif"], inplace=True)

    elif af_mode == "AF3":
        dataAF = pd.read_csv(path_AF / "AF3_output.tsv", sep="\t")

        benchmark_set_replace_dict = {"known extension": "known_extension", "random extension" : "random_extension"}
            
        for i, row in dataAF.iterrows():
            benchmark_set = benchmark_set_replace_dict[row["benchmark_set"]]
            dataAF.at[i, "benchmark_set"] = benchmark_set
enhance_dataframe()
display(dataAF)

['prediction_name', 'chainA_length', 'chainB_length', 'model_id', 'model_confidence', 'chainA_intf_avg_plddt', 'chainB_intf_avg_plddt', 'intf_avg_plddt', 'pDockQ', 'iPAE', 'num_chainA_intf_res', 'num_chainB_intf_res', 'num_res_res_contact', 'num_atom_atom_contact', 'mutation_in_motif', 'label', 'prediction_type']
Column chainA_start not (yet) in data frame
Column chainA_end not (yet) in data frame
Column chainB_start not (yet) in data frame
Column chainB_end not (yet) in data frame
Column num_align_atoms_domain not (yet) in data frame
Column num_align_resi_domain not (yet) in data frame
Column hbonds not (yet) in data frame
Column salt_bridges not (yet) in data frame
Column hydrophobic_interactions not (yet) in data frame
Column disulfide_bonds not (yet) in data frame
Column run_id not (yet) in data frame
Column chainA_id not (yet) in data frame
Column chainB_id not (yet) in data frame
Column chainA_start not (yet) in data frame
Column chainA_end not (yet) in data frame
Column chainB_s

,benchmark_set,prediction_name,model_id,model_confidence,chainA_length,chainA_intf_avg_plddt,chainB_length,chainB_intf_avg_plddt,intf_avg_plddt,pDockQ,iPAE,num_chainA_intf_res,num_chainB_intf_res,num_res_res_contact,num_atom_atom_contact,prediction_type,num_mutations
0,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_0,0.722553,499,96.590455,25,87.037000,93.605000,0.278492,1.939236,22,10,43,352,motif_ext + domain_ext,<NA>
1,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_1,0.710868,499,96.486842,25,79.521250,91.460000,0.191685,2.697327,19,8,35,322,motif_ext + domain_ext,<NA>
2,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_2,0.706698,499,97.075000,25,88.953334,94.367778,0.237941,2.069981,18,9,39,359,motif_ext + domain_ext,<NA>
3,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_3,0.689345,499,96.403889,25,86.464444,93.090740,0.217522,2.071760,18,9,38,347,motif_ext + domain_ext,<NA>
4,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_4,0.679946,499,96.648889,25,84.273749,92.841154,0.213639,2.396510,18,8,36,339,motif_ext + domain_ext,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5640,random_extension,MLIG_WW_1_MFL.DLIG_PAM2_2_Dmin,ranked_0,0.202944,72,58.186667,895,30.820000,51.345000,0.018259,30.424984,3,1,3,25,motif_ext + domain_min,<NA>
5641,random_extension,MLIG_WW_1_MFL.DLIG_PAM2_2_Dmin,ranked_1,0.189991,72,0.000000,895,0.000000,0.000000,0.000000,50.000000,0,0,0,0,motif_ext + domain_min,<NA>
5642,random_extension,MLIG_WW_1_MFL.DLIG_PAM2_2_Dmin,ranked_2,0.187835,72,61.299999,895,74.949999,70.399999,0.021272,26.806385,1,2,2,5,motif_ext + domain_min,<NA>
5643,random_extension,MLIG_WW_1_MFL.DLIG_PAM2_2_Dmin,ranked_3,0.180747,72,0.000000,895,0.000000,0.000000,0.000000,50.000000,0,0,0,0,motif_ext + domain_min,<NA>


In [6]:
# Read in solved structure data

dataSolved = pd.DataFrame(columns=["set", "PDB_id", "DDI_pfam_id", "path", "chainA_id", "chainB_id"])

# DMI
for structure_file in [p for p in Path("DMI").iterdir() if p.is_file() and p.suffix == ".pdb"]:
    pdb_id = structure_file.name.split("_")[0]
    dataSolved.loc[len(dataSolved)] = {"set" : "DMI", "PDB_id": pdb_id, "path": structure_file.relative_to(path_solved), "chainA_id": "A", "chainB_id": "B"}

# DDI
for structure_file in [p for p in Path("DDI").iterdir() if p.is_file() and p.suffix == ".pdb"]:
    ddi_pfam_id = "_".join(structure_file.name.split("_")[0:2])
    pdb_id = structure_file.name.split("_")[2]
    chainA_id = structure_file.name.split("_")[3][0]
    chainB_id = structure_file.name.split("_")[3][1]
    dataSolved.loc[len(dataSolved)] = {"set" : "DDI", "PDB_id": pdb_id, "DDI_pfam_id": ddi_pfam_id, "path": structure_file.relative_to(path_solved), "chainA_id": chainA_id, "chainB_id": chainB_id}

display(dataSolved)

,set,PDB_id,DDI_pfam_id,path,chainA_id,chainB_id


### 1 Parsing the file names
Many informations (PDB ID, mutation sequence, ...) are included in the filename. This section parses them and adds them to the metrics data frame. The detected values include:
* **PDB_id** (all structures): RCSB Protein Data Bank ID
* **ELM_instance** (DMI): ID of the motif in the Eukaryotic Linear Motif database
* **DDI_pfam_id** (DDI): ID of the two domains in the Pfam Database separated by an underscore
* **PDB_id_random_paired**, **ELM_instance_random_paired** and **DDI_pfam_id_random_paired** (random_DMI, random_DDI): The id of the second domain/motif for the randomly paired DMI and DDI. For DMI, the random fields always encode the motif.
* **sequence_initial** and **sequence_mutated** (mutations_DMI): For the mutations, this fields encode the initinal and mutated sequence of the motif
* **chainA_id**, **chainA_start**, **chainA_end** and the same three for **chainB** (all structures where data is available): The start and end residue as well as the ID of the chains in the experimentally solved structure. For known_DDI, the information is encoded in the prediction name and carried over to the random_DDI set. For DMI structures, the information will be added in section 2 by aligning with the template file. Chain A and B refer to the IDs in the AlphaFold predictions. For DMI, chain B is always the motif chain.

Note: known_extensions were excluded from the benchmark set, but if you need to parse them remove the comments in the code cell below. But the code needs to be tested !!

In [7]:
"""
# Regex patterns for the actual filename formats
import re

# Based on the actual examples:
# DEG_APCC_KENBOX_2_M15_M39_D1_D499 (known_extension - uppercase, with M/D positions)
# lig_pcna_tls_4_m814_m870_d1_d261 (random_extension - lowercase, with m/d positions)

regex_known_extension = r"^(.+)_M(\d+|min|fl)_M(\d+|min|fl)_D(\d+|min|fl)_D(\d+|min|fl)$"  # e.g., DEG_APCC_KENBOX_2_M15_M39_D1_D499
regex_random_extension = r"^(.+)_m(\d+|min|fl)_m(\d+|min|fl)_d(\d+|min|fl)_d(\d+|min|fl)$"  # e.g., any_name_m814_m870_d1_d261

# Initialize columns for extension data
dataAF["PDB_id"] = None
dataAF["ELM_instance"] = None
dataAF["PDB_id_random_paired"] = None
dataAF["ELM_instance_random_paired"] = None
dataAF["known_extension_motif"] = None 
dataAF["known_extension_domain"] = None
dataAF["chainA_id"] = None
dataAF["chainB_id"] = None
dataAF["chainA_start"] = None
dataAF["chainA_end"] = None
dataAF["chainB_start"] = None
dataAF["chainB_end"] = None

print("Processing extension samples...")

successful_matches = 0
failed_matches = []

for i, row in dataAF.iterrows():
    if row["benchmark_set"] not in ["known_extension", "random_extension"]:
        continue
        
    pdb_id, pdb_id_2, elm_instance, elm_instance_2 = None, None, None, None
    known_extensionM, known_extensionD, chain1_letter, chain2_letter = None, None, None, None
    c1_start, c1_end, c2_start, c2_end = None, None, None, None
    
    benchmark_set = row["benchmark_set"]
    prediction_name = row["prediction_name"]
    
    if benchmark_set == "known_extension":
        # Pattern: DEG_APCC_KENBOX_2_M15_M39_D1_D499 (uppercase M/D positions)
        match = re.search(regex_known_extension, prediction_name)
        if match and len(match.groups()) == 5:
            elm_instance = match.groups()[0]  # Base name: "DEG_APCC_KENBOX_2"
            m_start = match.groups()[1]       # Motif start: "15" or "min" or "fl"
            m_end = match.groups()[2]         # Motif end: "39" or "min" or "fl"
            d_start = match.groups()[3]       # Domain start: "1" or "min" or "fl"
            d_end = match.groups()[4]         # Domain end: "499" or "min" or "fl"
            
            # Convert to integers if they're numeric, otherwise keep as None
            try:
                c1_start = int(m_start) if m_start.isdigit() else None
                c1_end = int(m_end) if m_end.isdigit() else None
                c2_start = int(d_start) if d_start.isdigit() else None
                c2_end = int(d_end) if d_end.isdigit() else None
            except ValueError:
                c1_start = c1_end = c2_start = c2_end = None
            
            chain1_letter, chain2_letter = "A", "B"
            successful_matches += 1
        else:
            failed_matches.append((benchmark_set, prediction_name))
    
    elif benchmark_set == "random_extension":
        # Pattern: any_name_m814_m870_d1_d261 (lowercase m/d positions)
        match = re.search(regex_random_extension, prediction_name)
        if match and len(match.groups()) == 5:
            elm_instance = match.groups()[0]  # Base name: everything before _m..._m..._d..._d...
            m_start = match.groups()[1]       # Motif start: "814" or "min" or "fl"
            m_end = match.groups()[2]         # Motif end: "870" or "min" or "fl"
            d_start = match.groups()[3]       # Domain start: "1" or "min" or "fl"
            d_end = match.groups()[4]         # Domain end: "261" or "min" or "fl"
            
            # Convert to integers if they're numeric, otherwise keep as None
            try:
                c1_start = int(m_start) if m_start.isdigit() else None
                c1_end = int(m_end) if m_end.isdigit() else None
                c2_start = int(d_start) if d_start.isdigit() else None
                c2_end = int(d_end) if d_end.isdigit() else None
            except ValueError:
                c1_start = c1_end = c2_start = c2_end = None
            
            chain1_letter, chain2_letter = "A", "B"
            successful_matches += 1
        else:
            failed_matches.append((benchmark_set, prediction_name))
    
    # Update the dataframe
    dataAF.at[i, "PDB_id"] = pdb_id
    dataAF.at[i, "PDB_id_random_paired"] = pdb_id_2
    dataAF.at[i, "ELM_instance"] = elm_instance
    dataAF.at[i, "ELM_instance_random_paired"] = elm_instance_2
    dataAF.at[i, "known_extension_motif"] = known_extensionM
    dataAF.at[i, "known_extension_domain"] = known_extensionD
    dataAF.at[i, "chainA_id"] = chain1_letter
    dataAF.at[i, "chainB_id"] = chain2_letter
    dataAF.at[i, "chainA_start"] = c1_start
    dataAF.at[i, "chainA_end"] = c1_end
    dataAF.at[i, "chainB_start"] = c2_start
    dataAF.at[i, "chainB_end"] = c2_end

# Filter to only keep extension samples
dataAF = dataAF[dataAF["benchmark_set"].isin(["known_extension", "random_extension"])]

print(f"\nMatching Results:")
print(f"Successful matches: {successful_matches}")
print(f"Failed matches: {len(failed_matches)}")

if failed_matches:
    print(f"\nFailed matches:")
    for benchmark_set, filename in failed_matches[:10]:  # Show first 10 failures
        print(f"  {benchmark_set}: {filename}")
    if len(failed_matches) > 10:
        print(f"  ... and {len(failed_matches) - 10} more")

print(f"\nTotal rows after filtering: {len(dataAF)}")
if len(dataAF) > 0:
    print("\nSample processed data:")
    display(dataAF.head())

"""

'\n# Regex patterns for the actual filename formats\nimport re\n\n# Based on the actual examples:\n# DEG_APCC_KENBOX_2_M15_M39_D1_D499 (known_extension - uppercase, with M/D positions)\n# lig_pcna_tls_4_m814_m870_d1_d261 (random_extension - lowercase, with m/d positions)\n\nregex_known_extension = r"^(.+)_M(\\d+|min|fl)_M(\\d+|min|fl)_D(\\d+|min|fl)_D(\\d+|min|fl)$"  # e.g., DEG_APCC_KENBOX_2_M15_M39_D1_D499\nregex_random_extension = r"^(.+)_m(\\d+|min|fl)_m(\\d+|min|fl)_d(\\d+|min|fl)_d(\\d+|min|fl)$"  # e.g., any_name_m814_m870_d1_d261\n\n# Initialize columns for extension data\ndataAF["PDB_id"] = None\ndataAF["ELM_instance"] = None\ndataAF["PDB_id_random_paired"] = None\ndataAF["ELM_instance_random_paired"] = None\ndataAF["known_extension_motif"] = None \ndataAF["known_extension_domain"] = None\ndataAF["chainA_id"] = None\ndataAF["chainB_id"] = None\ndataAF["chainA_start"] = None\ndataAF["chainA_end"] = None\ndataAF["chainB_start"] = None\ndataAF["chainB_end"] = None\n\nprint("Proce

In [18]:
# Simplified regex pattern for extension samples
import re

# Pattern for known extensions - handles all the variations we've seen
known_extension_pattern = r"^(.+?)_[Mm](\d+|min|fl|FL|Min|Fl)(?:_[Mm](\d+|min|fl|FL|Min|Fl))?_[Dd](\d+|min|fl|FL|Min|Fl)(?:_[Dd](\d+|min|fl|FL|Min|Fl))?$"

def parse_known_extension_name(prediction_name):
    """Parse known extension prediction name and return extracted components"""
    match = re.search(known_extension_pattern, prediction_name)
    if not match:
        return None, None, None, None, None, None, None
    
    groups = match.groups()
    elm_instance = groups[0]  # Base name
    
    # Handle motif positions (M/m) - first is always present
    m_start = groups[1] if groups[1] else None
    m_end = groups[2] if groups[2] else None  # May be None for single M format
    
    # Handle domain positions (D/d) - first is always present  
    d_start = groups[3] if groups[3] else None
    d_end = groups[4] if groups[4] else None  # May be None for single D format
    
    # Convert numeric strings to integers
    def convert_to_int(value):
        if value and value.isdigit():
            return int(value)
        return None
    
    c1_start = convert_to_int(m_start)
    c1_end = convert_to_int(m_end)
    c2_start = convert_to_int(d_start)
    c2_end = convert_to_int(d_end)
    
    return elm_instance, c1_start, c1_end, c2_start, c2_end, "A", "B"

# Initialize columns for known extensions only
columns_to_add = [
    "ELM_instance", "chainA_id", "chainB_id",
    "chainA_start", "chainA_end", "chainB_start", "chainB_end"
]

for col in columns_to_add:
    if col not in dataAF.columns:
        dataAF[col] = None

print("Processing known extension samples only...")

successful_matches = 0
failed_matches = []

# Process only known extension samples
known_extension_mask = dataAF["benchmark_set"] == "known_extension"
known_extension_data = dataAF[known_extension_mask]

for i, row in known_extension_data.iterrows():
    prediction_name = row["prediction_name"]
    
    # Parse the prediction name
    elm_instance, c1_start, c1_end, c2_start, c2_end, chain1, chain2 = parse_known_extension_name(prediction_name)
    
    if elm_instance is not None:
        # Update the dataframe
        dataAF.at[i, "ELM_instance"] = elm_instance
        dataAF.at[i, "chainA_id"] = chain1
        dataAF.at[i, "chainB_id"] = chain2
        dataAF.at[i, "chainA_start"] = c1_start
        dataAF.at[i, "chainA_end"] = c1_end
        dataAF.at[i, "chainB_start"] = c2_start
        dataAF.at[i, "chainB_end"] = c2_end
        
        successful_matches += 1
    else:
        failed_matches.append(prediction_name)

# Filter to only keep known extension samples
dataAF = dataAF[known_extension_mask]

print(f"\nMatching Results for Known Extensions:")
print(f"Successful matches: {successful_matches}")
print(f"Failed matches: {len(failed_matches)}")

if failed_matches:
    print(f"\nFailed matches:")
    for filename in failed_matches[:20]:  # Show first 20 failures
        print(f"  {filename}")
    if len(failed_matches) > 20:
        print(f"  ... and {len(failed_matches) - 20} more")

print(f"\nTotal known extension rows: {len(dataAF)}")
if len(dataAF) > 0:
    print("\nSample processed data:")
    display(dataAF.head())

display(dataAF)


Processing known extension samples only...

Matching Results for Known Extensions:
Successful matches: 2815
Failed matches: 0

Total known extension rows: 2815

Sample processed data:


,benchmark_set,prediction_name,model_id,model_confidence,chainA_length,chainA_intf_avg_plddt,chainB_length,chainB_intf_avg_plddt,intf_avg_plddt,pDockQ,...,num_align_atoms_domain,num_align_resi_domain,RMSD_all_atom,RMSD_domain,RMSD_backbone_peptide,RMSD_all_atom_peptide,DockQ,iRMSD,LRMSD,Fnonnat
0,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_0,0.722553,499,96.590455,25,87.037000,93.605000,0.278492,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
1,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_1,0.710868,499,96.486842,25,79.521250,91.460000,0.191685,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_2,0.706698,499,97.075000,25,88.953334,94.367778,0.237941,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
3,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_3,0.689345,499,96.403889,25,86.464444,93.090740,0.217522,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
4,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_4,0.679946,499,96.648889,25,84.273749,92.841154,0.213639,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN


,benchmark_set,prediction_name,model_id,model_confidence,chainA_length,chainA_intf_avg_plddt,chainB_length,chainB_intf_avg_plddt,intf_avg_plddt,pDockQ,...,num_align_atoms_domain,num_align_resi_domain,RMSD_all_atom,RMSD_domain,RMSD_backbone_peptide,RMSD_all_atom_peptide,DockQ,iRMSD,LRMSD,Fnonnat
0,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_0,0.722553,499,96.590455,25,87.037000,93.605000,0.278492,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
1,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_1,0.710868,499,96.486842,25,79.521250,91.460000,0.191685,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_2,0.706698,499,97.075000,25,88.953334,94.367778,0.237941,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
3,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_3,0.689345,499,96.403889,25,86.464444,93.090740,0.217522,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
4,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_4,0.679946,499,96.648889,25,84.273749,92.841154,0.213639,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2810,known_extension,TRG_AP2beta_CARGO_1_Mmin_DFL,ranked_0,0.713720,937,88.416363,11,57.284286,76.309444,0.024013,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2811,known_extension,TRG_AP2beta_CARGO_1_Mmin_DFL,ranked_1,0.310416,937,66.480000,11,24.590000,49.724000,0.018259,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2812,known_extension,TRG_AP2beta_CARGO_1_Mmin_DFL,ranked_2,0.209493,937,0.000000,11,0.000000,0.000000,0.000000,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2813,known_extension,TRG_AP2beta_CARGO_1_Mmin_DFL,ranked_3,0.201790,937,0.000000,11,0.000000,0.000000,0.000000,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN


### 2 Adding domain and motif start / end from template file
While for the DDI structures selection start and end are included in the filename, for DMI structures there is absolutely no information about start/end of motif and domain. At least, the DMI structures are cut to only include the minimal domain/motif, but there still may be mutations or missing residues in experimental structures. To restore this information use the template and perform a simple search for three consecutive residues in both chains and calculate the offset between the chain IDs. Then take the most common offset and use it if at least 50 % of the AF residues were matched this way.

In [9]:
def align_sequences(chain_af:  Chain, chain_template: Chain) -> tuple[int, int, float, str, str]:
    """ Estimate the residue id offset between two chains based on a neighbour local alignment (BioPython has no convinient alignment function).
    
        :returns tuple[int, int, float, str, str]: Start ID, End ID, score, Sequence Chain A, Sequence Chain B
    """
    residues_af = [r for r in chain_af.get_residues()]
    residues_tpl = [r for r in chain_template.get_residues()]
    seq_af = seq1(''.join([r.resname for r in residues_af]))
    seq_tpl = seq1(''.join([r.resname for r in residues_tpl]))
    offset_list = []

    misscounts = 0
    for a0, a1, a2 in zip(residues_af[:-2], residues_af[1:-1], residues_af[2:]):
        _found = False
        for t0, t1, t2 in zip(residues_tpl[:-2], residues_tpl[1:-1], residues_tpl[2:]):
            if a0.resname == t0.resname and a1.resname == t1.resname and a2.resname == t2.resname:
                offset_list.append(t1.id[1] - a1.id[1])
                _found = True
        if not _found:
            misscounts += 1

    # For degenerated short chains (motif) use no neighbours for matching
    # if len(offset_list) == 0:
    #     for r1 in residues_af:
    #         for r2 in residues_tpl:
    #             if r1.resname == r2.resname:
    #                 offset_list.append(r2.id[1] - r1.id[1])

    if len(offset_list) == 0:
        return (None, None, 0, seq_af, seq_tpl)
    offsets, counts = np.unique(offset_list, return_counts=True)
    offset = offsets[np.argmax(counts)]
    score = 1 - misscounts/(len(residues_af) - 2)
    return  offset + 1, offset + len(residues_af), score, seq_af, seq_tpl

for i, row in dataAF[dataAF["benchmark_set"].isin(["known_DMI", "random_DMI", "mutations_DMI"])].iterrows():
    pdb_id = str(row["PDB_id"])
    pdb_id_2 = None
    if row["PDB_id_random_paired"] is not None:
        pdb_id_2 = str(row["PDB_id_random_paired"])
    prediction_name = row["prediction_name"]
    benchmark_set = row["benchmark_set"]
    model_id = row["model_id"]

    if model_id == "ranked_0":
        print(bcolors.OKBLUE + f"{prediction_name} ({benchmark_set})" + bcolors.ENDC)

    #if not prediction_name == "MLIG_MYND_2_2ODD.DMOD_SUMO_for_1_1KPS": continue

    af_path = path_AF / "DMI" / benchmark_set / prediction_name / (model_id + ".pdb")
    af_biopy = parser.get_structure("structure", file=af_path)[0]
    chainA_af = af_biopy["A"]
    chainB_af = af_biopy["B"]    

    template1_path = path_solved / "DMI" / (pdb_id + "_min_DMI.pdb")
    if not template1_path.exists():
        print(f"\t", bcolors.WARNING + f"{prediction_name} has no template file for {pdb_id}" + bcolors.WARNING)
        continue
    template1_biopy = parser.get_structure("structure", file=template1_path)[0]
    chainA_tlp = template1_biopy["A"]
    if pdb_id_2 is not None:
        template2_path = path_solved / "DMI" / (pdb_id_2 + "_min_DMI.pdb")
        if not template2_path.exists():
            print(f"\t", f"{prediction_name} has no template file for {pdb_id}")
            continue
        template2_biopy = parser.get_structure("structure", file=template2_path)[0]
        chainB_tlp = template2_biopy["B"]
    else:
        chainB_tlp = template1_biopy["B"]

    chainA_start, chainA_end, chainA_score, seqA_af, seqA_tpl = align_sequences(chain_af=chainA_af, chain_template=chainA_tlp)
    if chainA_start is not None:
        if model_id == "ranked_0":
            print("\t", f"chainA: {chainA_start}-{chainA_end} ({bcolors.WARNING if chainA_score < 0.5 else ''}{chainA_score:0.3f}{bcolors.ENDC})")
        dataAF.at[i, "chainA_start"] =  chainA_start
        dataAF.at[i, "chainA_end"] =  chainA_end
    else:
        if model_id == "ranked_0":
            print(f"\t", bcolors.WARNING + "Chain A alignment failed" + bcolors.ENDC)
    if model_id == "ranked_0" and chainA_score < 0.5:
        print("\t\t", seqA_af)
        print("\t\t", seqA_tpl)

    chainB_start, chainB_end, chainB_score, seqB_af, seqB_tpl = align_sequences(chain_af=chainB_af, chain_template=chainB_tlp)
    if chainB_start is not None:
        if model_id == "ranked_0":
            print("\t", f"chainB: {chainB_start}-{chainB_end} ({bcolors.WARNING if chainB_score < 0.5 else ''}{chainB_score:0.3f}{bcolors.ENDC})")
        dataAF.at[i, "chainB_start"] =  chainB_start
        dataAF.at[i, "chainB_end"] =  chainB_end
    else:
        if model_id == "ranked_0":
            print(f"\t", bcolors.WARNING + "Chain B alignment failed" + bcolors.ENDC)
    if model_id == "ranked_0" and chainB_score < 0.5:
        print("\t\t", seqB_af)
        print("\t\t", seqB_tpl)
    
    

# For the mutations, the alignment mostly fails. For those restore the information using the known_DMI dataset
for i, row in dataAF[dataAF["benchmark_set"].isin(["mutations_DMI"])].iterrows():
    prediction_name = row["prediction_name"]
    benchmark_set = row["benchmark_set"]
    pdb_id = row["PDB_id"]
    pdb_id_2 = row["PDB_id_random_paired"] if row["PDB_id_random_paired"] is not None else pdb_id
    
    if len(list((_row1 := dataAF[np.logical_and(dataAF["benchmark_set"] == "known_DMI", dataAF["PDB_id"] == pdb_id)])["chainA_id"])) == 0:
        if model_id == "ranked_0":
            print(f"Can't find {pdb_id} from {prediction_name} ({benchmark_set}, chain A) in the known_DMI set")
        continue
    if len(list((_row2 := dataAF[np.logical_and(dataAF["benchmark_set"] == "known_DMI", dataAF["PDB_id"] == pdb_id_2)])["chainB_id"])) == 0:
        if model_id == "ranked_0":
            print(f"Can't find {pdb_id_2} from {prediction_name} ({benchmark_set}, chain B) in the known_DMI set")
        continue
    dataAF.at[i, "chainA_start"] = list(_row1["chainA_start"])[0]
    dataAF.at[i, "chainA_end"] = list(_row1["chainA_end"])[0]
    dataAF.at[i, "chainB_start"] = list(_row2["chainB_start"])[0]
    dataAF.at[i, "chainB_end"] = list(_row2["chainB_end"])[0]

In [10]:
dataAF[np.logical_and(dataAF["chainA_end"].isna(), dataAF["model_id"] == "ranked_0")]

,benchmark_set,prediction_name,model_id,model_confidence,chainA_length,chainA_intf_avg_plddt,chainB_length,chainB_intf_avg_plddt,intf_avg_plddt,pDockQ,...,num_atom_atom_contact,prediction_type,num_mutations,ELM_instance,chainA_id,chainB_id,chainA_start,chainA_end,chainB_start,chainB_end
50,known_extension,DEG_APCC_KENBOX_2_Mmin_D1_D499,ranked_0,0.885199,499,96.832666,5,88.506000,94.750999,0.196473,...,199,motif_min + domain_ext,<NA>,DEG_APCC_KENBOX_2,A,B,None,None,1,499
55,known_extension,DEG_APCC_KENBOX_2_Mmin_DFL,ranked_0,0.886823,499,96.778666,5,89.178001,94.878500,0.185877,...,199,motif_min + domain_ext,<NA>,DEG_APCC_KENBOX_2,A,B,None,None,None,None
120,known_extension,DEG_Kelch_Keap1_1_Mmin_D175_D624,ranked_0,0.936616,450,98.606842,6,88.848334,96.264800,0.325345,...,269,motif_min + domain_ext,<NA>,DEG_Kelch_Keap1_1,A,B,None,None,175,624
125,known_extension,DEG_Kelch_Keap1_1_Mmin_D295_D624,ranked_0,0.934373,330,98.708333,6,92.824998,97.237499,0.338935,...,270,motif_min + domain_ext,<NA>,DEG_Kelch_Keap1_1,A,B,None,None,295,624
130,known_extension,DEG_Kelch_Keap1_1_Mmin_DFL,ranked_0,0.908973,624,98.520000,6,83.040000,94.804800,0.293742,...,266,motif_min + domain_ext,<NA>,DEG_Kelch_Keap1_1,A,B,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2765,known_extension,LIG_Vh1_VBS_1_Mmin_D1_D925,ranked_0,0.911675,925,89.096562,19,88.291176,88.817143,0.530499,...,349,motif_min + domain_ext,<NA>,LIG_Vh1_VBS_1,A,B,None,None,1,925
2770,known_extension,LIG_Vh1_VBS_1_Mmin_DFL,ranked_0,0.880064,1134,86.502352,19,82.836471,85.280392,0.476883,...,373,motif_min + domain_ext,<NA>,LIG_Vh1_VBS_1,A,B,None,None,None,None
2775,known_extension,LIG_WW_1_MFL_Dmin,ranked_0,0.522571,256,81.343928,895,29.278387,62.791839,0.208710,...,1618,motif_ext + domain_min,<NA>,LIG_WW_1,A,B,None,None,None,None
2805,known_extension,TRG_AP2beta_CARGO_1_Mmin_D4_D937,ranked_0,0.703279,934,88.632728,11,56.381667,77.250001,0.022942,...,100,motif_min + domain_ext,<NA>,TRG_AP2beta_CARGO_1,A,B,None,None,4,937


### 3 RMSD with PyMOL

For all structures, calculate the overall RMSD
- **RMSD_all_atom**: RMSD aligning the whole structure

For DMI, align the domains (chain A) first
- **align_score_domain**: Score of domain alignment
- **num_align_atoms_domain** and **num_align_resi_domain**: Count of aligned atoms/residues of domain
- **RMSD_domain**: RMSD of the domain (chain A) after aligning on the domain
For known_DMI and mutations_DMI (excluding the mutated residues):
- **RMSD_backbone_peptide** and **RMSD_all_atom_peptide**: RMSD of the motif chain (chain B) after aligning on the domain

For DDI perform, use the longest chain (or chain A if both have equal length) as domain and define the shorter one as peptide. Then use the same definition as for DMI

In [19]:
# Calculating the RMSD related values using pymol

dataAF["align_score_domain"] = None
dataAF["num_align_atoms_domain"] = None
dataAF["num_align_resi_domain"] = None
dataAF["RMSD_all_atom"] = None
dataAF["RMSD_domain"] = None
dataAF["RMSD_backbone_peptide"] = None
dataAF["RMSD_all_atom_peptide"] = None

for i,row in dataAF.iterrows():
    benchmark_set = str(row["benchmark_set"])
    _set = "DDI" if "DDI" in benchmark_set else "DMI"
    pdb_id = str(row["PDB_id"]) if row.notnull()["PDB_id"] else None
    pdb_id_2 = str(row["PDB_id_random_paired"]) if row.notnull()["PDB_id_random_paired"] else None
    ddi_pfam_id = str(row["DDI_pfam_id"]) if row.notnull()["DDI_pfam_id"] else None
    ddi_pfam_id_2 = str(row["DDI_pfam_id_random_paired"]) if row.notnull()["DDI_pfam_id_random_paired"] else None
    prediction_name = str(row["prediction_name"]) if row.notnull()["prediction_name"] else None
    model_id = str(row["model_id"]) if row.notnull()["model_id"] else None
    chainA_id = str(row["chainA_id"]) if row.notnull()["chainA_id"] else None
    chainB_id = str(row["chainB_id"]) if row.notnull()["chainB_id"] else None
    chainA_start = int(row["chainA_start"]) if row.notnull()["chainA_start"] else None
    chainB_start = int(row["chainB_start"]) if row.notnull()["chainB_start"] else None
    chainA_end = int(row["chainA_end"]) if row.notnull()["chainA_end"] else None
    chainB_end = int(row["chainB_end"]) if row.notnull()["chainB_end"] else None
    chainA_length = int(row["chainA_length"]) if row.notnull()["chainA_length"] else None
    chainB_length = int(row["chainB_length"]) if row.notnull()["chainB_length"] else None

    if model_id == "ranked_0":
        pymol.cmd.reinitialize() 
        print(f"{bcolors.OKBLUE}{prediction_name} ({benchmark_set}){bcolors.ENDC}")    

    structure_path = path_AF / _set / benchmark_set / prediction_name / (model_id + ".pdb")
    if not structure_path.exists():
        print(f"\t{bcolors.FAIL}{prediction_name} ({benchmark_set}) does not exist.{bcolors.ENDC} Skip RMSD calculation")
        continue
    
    template_row = dataSolved.loc[np.logical_and(dataSolved["set"] == _set, np.logical_and(dataSolved["PDB_id"] == pdb_id, np.logical_or(dataSolved["DDI_pfam_id"] == ddi_pfam_id, dataSolved["DDI_pfam_id"].isna())))]
    if len(template_row) == 0:
        print(f"\t{bcolors.FAIL}Can't find template structure for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id}.{bcolors.ENDC} Skip RMSD calculation")
        continue
    elif len(template_row) >= 2:
        print(f"\t{bcolors.FAIL}Multiple template structures found for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id}.{bcolors.ENDC} Skip RMSD calculation")
        continue
    template_path = path_solved / str(template_row["path"].item())

    template2_path = None
    if pdb_id_2 is not None:
        template2_row = dataSolved.loc[np.logical_and(dataSolved["set"] == _set, np.logical_and(dataSolved["PDB_id"] == pdb_id_2, np.logical_or(dataSolved["DDI_pfam_id"] == ddi_pfam_id_2, dataSolved["DDI_pfam_id"].isna())))]
        if len(template2_row) == 0:
            print(f"\t{bcolors.FAIL}Can't find template structure for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id_2}.{bcolors.ENDC} Skip RMSD calculation")
            continue
        elif len(template2_row) >= 2:
            print(f"\t{bcolors.FAIL}Multiple template structures found for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id_2}.{bcolors.ENDC} Skip RMSD calculation")
            continue

        template2_path = path_resources / "solved" / str(template2_row["path"].item())

    #pymol.cmd.reinitialize() # Not needed usually, but slows performance significantly down
    for o in pymol.cmd.get_object_list():
        pymol.cmd.delete("all")
    pymol.cmd.sort()

    # First loading the structures. Use two temporary objects to allow renaming the chains even if the chains have the same name or have switched IDs
    pymol.cmd.load(structure_path, "af")
    if template2_path is not None:
        # Updating the object is possible, but turned out to be unstable
        pymol.cmd.load(template_path, "solvedA")
        pymol.cmd.load(template2_path, "solvedB")
        pymol.cmd.create("solved1", f"solvedA and chain {chainA_id}")
        pymol.cmd.create("solved2", f"solvedB and chain {chainB_id}")
        pymol.cmd.delete("solvedA")
        pymol.cmd.delete("solvedB")
    else:
        pymol.cmd.load(template_path, "solvedraw")
        pymol.cmd.create("solved1", f"solvedraw and chain {chainA_id}")
        pymol.cmd.sort()
        pymol.cmd.create("solved2", f"solvedraw and chain {chainB_id}")
        pymol.cmd.delete("solvedraw")
    pymol.cmd.sort()
    # Now rename the chains and create merged object
    pymol.cmd.alter(f"solved1 and chain {chainA_id}", "chain = 'A'")
    pymol.cmd.sort()
    pymol.cmd.alter(f"solved2 and chain {chainB_id}", "chain = 'B'")
    pymol.cmd.sort()
    pymol.cmd.create("solved", f"solved1 or solved2")
    pymol.cmd.delete("solved1")
    pymol.cmd.delete("solved2")
    pymol.cmd.sort()

    # Remove hydrogens and hetatm
    #pymol.cmd.remove(selection="elem 'H' or hetatm")
    pymol.cmd.remove(selection="not backbone and not sidechain or elem 'H'")
    pymol.cmd.sort()

    # Remove alternate location identifiers
    pymol.cmd.remove("not alt ''+A") # Using +A syntax to only effect the atoms with an alternate location identifier set
    pymol.cmd.sort()
    pymol.cmd.alter("all", "alt=''")
    pymol.cmd.sort()

    # Slice the chains to the known start/end residues. For chain B and AF a reindexing is performed as the rms_cur cmd of pymol requires same residue numbers for alignment
    if chainA_start is not None and chainB_start is not None:
        pymol.cmd.create("solved", f"solved and ((chain A and resi {chainA_start}-{chainA_end}) or (chain B and resi {chainB_start}-{chainB_end}))", source_state=0, target_state=0)
        pymol.cmd.sort()
        offsetA = chainA_start - 1
        pymol.cmd.alter("af and chain A", f"resi = (int(resi) + {offsetA})")
        pymol.cmd.sort()

        offsetB = chainB_start - 1
        pymol.cmd.alter("af and chain B", f"resi = (int(resi) + {offsetB})")
        pymol.cmd.sort()
    else:
        print(f"\t{bcolors.FAIL}Can't find information about the chain start/end in the template.{bcolors.ENDC} This may lead to wrong RMSD peptide values, so skip")
        continue

    pymol.cmd.sort()

    # DDI
    chain_align_1, chain_align_2 = "A", "B"
    if _set == "DDI" and chainB_length > chainA_length:
        chain_align_1, chain_align_2 = "B", "A"

    #For debugging
    #space = {'solved_resi': [], "af_resi": []}
    #pymol.cmd.iterate("solved and chain B", "solved_resi.append(int(resi))", space=space)
    #pymol.cmd.iterate("af and chain B", "af_resi.append(int(resi))", space=space)

    #    0: RMSD after refinement
    #    1: Number of aligned atoms after refinement
    #    2: Number of refinement cycles
    #    3: RMSD before refinement
    #    4: Number of aligned atoms before refinement
    #    5: Raw alignment score
    #    6: Number of residues aligned
    # Cycles = 0 to prevent rejection of outliers
    align_output_1 = pymol.cmd.align(mobile=f"af and chain {chain_align_1}", target=f"solved and chain {chain_align_1}", object="algn_domain", cycles=0)
    pymol.cmd.sort()
    RMSD_domain = align_output_1[0]
    num_align_atoms_domain = align_output_1[1]
    align_score_domain = align_output_1[5]
    num_align_resi_domain = align_output_1[6]

    RMSD_backbone_peptide = pymol.cmd.rms_cur(mobile=f"af and chain {chain_align_2} and bb.", target=f"solved and chain {chain_align_2} and bb.", object="peptide_super_bb", cycles=0)
    RMSD_all_atom_peptide = pymol.cmd.rms_cur(mobile=f"af and chain {chain_align_2}", target=f"solved and chain {chain_align_2}", object="peptide_super_all_atoms", cycles=0)
    
    align_output_all = pymol.cmd.align(mobile="af", target="solved", object="algn_all", cycles=0, )
    RMSD_all_atoms = align_output_all[0]

    dataAF.at[i, "RMSD_domain"] =  RMSD_domain
    dataAF.at[i, "align_score_domain"] =  align_score_domain
    dataAF.at[i, "num_align_atoms_domain"] =  num_align_atoms_domain
    dataAF.at[i, "num_align_resi_domain"] =  num_align_resi_domain

    if "random" not in benchmark_set:
        dataAF.at[i, "RMSD_backbone_peptide"] =  RMSD_backbone_peptide
        dataAF.at[i, "RMSD_all_atom_peptide"] =  RMSD_all_atom_peptide

    dataAF.at[i, "RMSD_all_atom"] =  RMSD_all_atoms
        
display(dataAF)

KeyError: 'PDB_id'

In [14]:
# Calculating the RMSD related values using pymol - DMI only

import pandas as pd
import numpy as np
import pymol
from pathlib import Path

# Filter for DMI data only
dataAF_DMI = dataAF[dataAF["benchmark_set"].str.contains("DMI", na=False)].copy()

# Initialize RMSD columns
dataAF_DMI["align_score_domain"] = None
dataAF_DMI["num_align_atoms_domain"] = None
dataAF_DMI["num_align_resi_domain"] = None
dataAF_DMI["RMSD_all_atom"] = None
dataAF_DMI["RMSD_domain"] = None
dataAF_DMI["RMSD_backbone_peptide"] = None
dataAF_DMI["RMSD_all_atom_peptide"] = None

# Check available columns
print("Available columns in dataAF_DMI:")
print(dataAF_DMI.columns.tolist())
print(f"\nProcessing {len(dataAF_DMI)} DMI entries...")

for i, row in dataAF_DMI.iterrows():
    try:
        # Safely extract values with error handling
        benchmark_set = str(row.get("benchmark_set", ""))
        pdb_id = str(row.get("PDB_id", "")) if pd.notnull(row.get("PDB_id")) else None
        pdb_id_2 = str(row.get("PDB_id_random_paired", "")) if pd.notnull(row.get("PDB_id_random_paired")) else None
        ddi_pfam_id = str(row.get("DDI_pfam_id", "")) if pd.notnull(row.get("DDI_pfam_id")) else None
        ddi_pfam_id_2 = str(row.get("DDI_pfam_id_random_paired", "")) if pd.notnull(row.get("DDI_pfam_id_random_paired")) else None
        prediction_name = str(row.get("prediction_name", "")) if pd.notnull(row.get("prediction_name")) else None
        model_id = str(row.get("model_id", "")) if pd.notnull(row.get("model_id")) else None
        chainA_id = str(row.get("chainA_id", "")) if pd.notnull(row.get("chainA_id")) else None
        chainB_id = str(row.get("chainB_id", "")) if pd.notnull(row.get("chainB_id")) else None
        
        # Convert to int safely
        chainA_start = int(row.get("chainA_start")) if pd.notnull(row.get("chainA_start")) else None
        chainB_start = int(row.get("chainB_start")) if pd.notnull(row.get("chainB_start")) else None
        chainA_end = int(row.get("chainA_end")) if pd.notnull(row.get("chainA_end")) else None
        chainB_end = int(row.get("chainB_end")) if pd.notnull(row.get("chainB_end")) else None
        chainA_length = int(row.get("chainA_length")) if pd.notnull(row.get("chainA_length")) else None
        chainB_length = int(row.get("chainB_length")) if pd.notnull(row.get("chainB_length")) else None
        
        # Skip if essential data is missing
        if not all([prediction_name, model_id, chainA_id, chainB_id]):
            print(f"Skipping row {i}: Missing essential data")
            continue
            
        if model_id == "ranked_0":
            pymol.cmd.reinitialize() 
            print(f"Processing {prediction_name} ({benchmark_set})")    

        # Build structure path for DMI
        structure_path = path_AF / "DMI" / benchmark_set / prediction_name / (model_id + ".pdb")
        if not structure_path.exists():
            print(f"\t{prediction_name} ({benchmark_set}) does not exist. Skip RMSD calculation")
            continue
        
        # Find template structure
        template_condition = (
            (dataSolved["set"] == "DMI") & 
            (dataSolved["PDB_id"] == pdb_id) & 
            ((dataSolved["DDI_pfam_id"] == ddi_pfam_id) | dataSolved["DDI_pfam_id"].isna())
        )
        template_row = dataSolved.loc[template_condition]
        
        if len(template_row) == 0:
            print(f"\tCan't find template structure for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id}. Skip RMSD calculation")
            continue
        elif len(template_row) > 1:
            print(f"\tMultiple template structures found for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id}. Skip RMSD calculation")
            continue
            
        template_path = path_solved / str(template_row["path"].iloc[0])

        # Handle second template if exists
        template2_path = None
        if pdb_id_2 is not None and pdb_id_2 != "":
            template2_condition = (
                (dataSolved["set"] == "DMI") & 
                (dataSolved["PDB_id"] == pdb_id_2) & 
                ((dataSolved["DDI_pfam_id"] == ddi_pfam_id_2) | dataSolved["DDI_pfam_id"].isna())
            )
            template2_row = dataSolved.loc[template2_condition]
            
            if len(template2_row) == 0:
                print(f"\tCan't find template structure for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id_2}. Skip RMSD calculation")
                continue
            elif len(template2_row) > 1:
                print(f"\tMultiple template structures found for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id_2}. Skip RMSD calculation")
                continue
            
            template2_path = path_resources / "solved" / str(template2_row["path"].iloc[0])

        # Clear PyMOL objects
        pymol.cmd.delete("all")
        pymol.cmd.sort()

        # Load structures
        pymol.cmd.load(str(structure_path), "af")
        
        if template2_path is not None:
            pymol.cmd.load(str(template_path), "solvedA")
            pymol.cmd.load(str(template2_path), "solvedB")
            pymol.cmd.create("solved1", f"solvedA and chain {chainA_id}")
            pymol.cmd.create("solved2", f"solvedB and chain {chainB_id}")
            pymol.cmd.delete("solvedA")
            pymol.cmd.delete("solvedB")
        else:
            pymol.cmd.load(str(template_path), "solvedraw")
            pymol.cmd.create("solved1", f"solvedraw and chain {chainA_id}")
            pymol.cmd.create("solved2", f"solvedraw and chain {chainB_id}")
            pymol.cmd.delete("solvedraw")
        
        pymol.cmd.sort()
        
        # Rename chains and create merged object
        pymol.cmd.alter(f"solved1 and chain {chainA_id}", "chain = 'A'")
        pymol.cmd.alter(f"solved2 and chain {chainB_id}", "chain = 'B'")
        pymol.cmd.create("solved", "solved1 or solved2")
        pymol.cmd.delete("solved1")
        pymol.cmd.delete("solved2")
        pymol.cmd.sort()

        # Clean structures
        pymol.cmd.remove("not backbone and not sidechain or elem 'H'")
        pymol.cmd.remove("not alt ''+A")
        pymol.cmd.alter("all", "alt=''")
        pymol.cmd.sort()

        # Slice chains to known residue ranges
        if all([chainA_start, chainB_start, chainA_end, chainB_end]):
            pymol.cmd.create("solved", f"solved and ((chain A and resi {chainA_start}-{chainA_end}) or (chain B and resi {chainB_start}-{chainB_end}))", source_state=0, target_state=0)
            
            # Reindex AF structure to match template numbering
            offsetA = chainA_start - 1
            offsetB = chainB_start - 1
            pymol.cmd.alter("af and chain A", f"resi = (int(resi) + {offsetA})")
            pymol.cmd.alter("af and chain B", f"resi = (int(resi) + {offsetB})")
            pymol.cmd.sort()
        else:
            print(f"\tCan't find chain start/end information. Skip RMSD calculation")
            continue

        # For DMI: determine which chain is the domain (longer) and which is the motif (shorter)
        chain_domain = "A" if chainA_length >= chainB_length else "B"
        chain_motif = "B" if chain_domain == "A" else "A"
        
        print(f"\tDomain chain: {chain_domain} (length: {chainA_length if chain_domain == 'A' else chainB_length})")
        print(f"\tMotif chain: {chain_motif} (length: {chainB_length if chain_motif == 'B' else chainA_length})")

        # Calculate RMSD values
        # Domain alignment (align by domain chain)
        align_output_domain = pymol.cmd.align(
            mobile=f"af and chain {chain_domain}", 
            target=f"solved and chain {chain_domain}", 
            object="algn_domain", 
            cycles=0
        )
        
        RMSD_domain = align_output_domain[0]
        num_align_atoms_domain = align_output_domain[1]
        align_score_domain = align_output_domain[5]
        num_align_resi_domain = align_output_domain[6]

        # Motif/peptide RMSD (after domain alignment)
        RMSD_backbone_peptide = pymol.cmd.rms_cur(
            mobile=f"af and chain {chain_motif} and bb.", 
            target=f"solved and chain {chain_motif} and bb.", 
            object="peptide_super_bb", 
            cycles=0
        )
        
        RMSD_all_atom_peptide = pymol.cmd.rms_cur(
            mobile=f"af and chain {chain_motif}", 
            target=f"solved and chain {chain_motif}", 
            object="peptide_super_all_atoms", 
            cycles=0
        )
        
        # All-atom RMSD
        align_output_all = pymol.cmd.align(
            mobile="af", 
            target="solved", 
            object="algn_all", 
            cycles=0
        )
        RMSD_all_atoms = align_output_all[0]

        # Store results
        dataAF_DMI.at[i, "RMSD_domain"] = RMSD_domain
        dataAF_DMI.at[i, "align_score_domain"] = align_score_domain
        dataAF_DMI.at[i, "num_align_atoms_domain"] = num_align_atoms_domain
        dataAF_DMI.at[i, "num_align_resi_domain"] = num_align_resi_domain
        dataAF_DMI.at[i, "RMSD_all_atom"] = RMSD_all_atoms
        
        # Only calculate peptide RMSD for non-random sets
        if "random" not in benchmark_set.lower():
            dataAF_DMI.at[i, "RMSD_backbone_peptide"] = RMSD_backbone_peptide
            dataAF_DMI.at[i, "RMSD_all_atom_peptide"] = RMSD_all_atom_peptide
        
        print(f"\tCompleted: Domain RMSD = {RMSD_domain:.3f}, Peptide RMSD = {RMSD_backbone_peptide:.3f}")
        
    except Exception as e:
        print(f"Error processing row {i}: {str(e)}")
        continue

print(f"\nCompleted processing {len(dataAF_DMI)} DMI entries")
print("\nRMSD Results Summary:")
print(dataAF_DMI[["prediction_name", "RMSD_domain", "RMSD_backbone_peptide", "RMSD_all_atom"]].describe())

# Display results
display(dataAF_DMI)

Available columns in dataAF_DMI:
['benchmark_set', 'prediction_name', 'model_id', 'model_confidence', 'chainA_length', 'chainA_intf_avg_plddt', 'chainB_length', 'chainB_intf_avg_plddt', 'intf_avg_plddt', 'pDockQ', 'iPAE', 'num_chainA_intf_res', 'num_chainB_intf_res', 'num_res_res_contact', 'num_atom_atom_contact', 'prediction_type', 'num_mutations', 'ELM_instance', 'chainA_id', 'chainB_id', 'chainA_start', 'chainA_end', 'chainB_start', 'chainB_end', 'align_score_domain', 'num_align_atoms_domain', 'num_align_resi_domain', 'RMSD_all_atom', 'RMSD_domain', 'RMSD_backbone_peptide', 'RMSD_all_atom_peptide']

Processing 0 DMI entries...

Completed processing 0 DMI entries

RMSD Results Summary:
       prediction_name RMSD_domain RMSD_backbone_peptide RMSD_all_atom
count                0           0                     0             0
unique               0           0                     0             0
top                NaN         NaN                   NaN           NaN
freq               

,benchmark_set,prediction_name,model_id,model_confidence,chainA_length,chainA_intf_avg_plddt,chainB_length,chainB_intf_avg_plddt,intf_avg_plddt,pDockQ,...,chainA_end,chainB_start,chainB_end,align_score_domain,num_align_atoms_domain,num_align_resi_domain,RMSD_all_atom,RMSD_domain,RMSD_backbone_peptide,RMSD_all_atom_peptide


### 4 DockQ
Calculate DockQ metrics for the known_DMI and known_DDI set using the offical package. The following columns are added to the table:
- **DockQ**: DockQ metric
- **iRMSD**: RMSD of interfacial residues
- **LRMSD**: Ligand RMSD. In case of DMI is this the motif, for DDI the smaller domain
- **Fnonnat**: Fraction of predicted contacts that are not native (same as FPR)

For more details read the details on the offical GitHub repo: [https://github.com/bjornwallner/DockQ](https://github.com/bjornwallner/DockQ)

In [15]:
from DockQ.DockQ import load_PDB, run_on_all_native_interfaces

dataAF["DockQ"] = np.nan
dataAF["iRMSD"] = np.nan
dataAF["LRMSD"] = np.nan
dataAF["Fnonnat"] = np.nan
for i, row in dataAF[dataAF["benchmark_set"].isin(["known_DMI", "known_DDI"])].iterrows():
    benchmark_set = str(row["benchmark_set"])
    _set = "DDI" if "DDI" in benchmark_set else "DMI"
    pdb_id = str(row["PDB_id"]) if row.notnull()["PDB_id"] else None
    pdb_id_2 = str(row["PDB_id_random_paired"]) if row.notnull()["PDB_id_random_paired"] else None
    ddi_pfam_id = str(row["DDI_pfam_id"]) if row.notnull()["DDI_pfam_id"] else None
    ddi_pfam_id_2 = str(row["DDI_pfam_id_random_paired"]) if row.notnull()["DDI_pfam_id_random_paired"] else None
    prediction_name = str(row["prediction_name"]) if row.notnull()["prediction_name"] else None
    model_id = str(row["model_id"]) if row.notnull()["model_id"] else None
    chainA_id = str(row["chainA_id"]) if row.notnull()["chainA_id"] else None
    chainB_id = str(row["chainB_id"]) if row.notnull()["chainB_id"] else None
    chainA_start = int(row["chainA_start"]) if row.notnull()["chainA_start"] else None
    chainB_start = int(row["chainB_start"]) if row.notnull()["chainB_start"] else None
    chainA_end = int(row["chainA_end"]) if row.notnull()["chainA_end"] else None
    chainB_end = int(row["chainB_end"]) if row.notnull()["chainB_end"] else None

    if model_id == "ranked_0":
        print(f"{bcolors.OKBLUE}{prediction_name} ({benchmark_set}){bcolors.ENDC}")

    structure_path = path_AF / _set / benchmark_set / prediction_name / (model_id + ".pdb")
    if not structure_path.exists():
        print(f"\t{bcolors.FAIL}{prediction_name} ({benchmark_set}) does not exist.{bcolors.ENDC} Skip DockQ")
        continue

    template_row = dataSolved.loc[np.logical_and(dataSolved["set"] == _set, np.logical_and(dataSolved["PDB_id"] == pdb_id, np.logical_or(dataSolved["DDI_pfam_id"] == ddi_pfam_id, dataSolved["DDI_pfam_id"].isna())))]
    if len(template_row) == 0:
        print(f"\t{bcolors.FAIL}Can't find template structure for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id}.{bcolors.ENDC} Skip")
        continue
    elif len(template_row) >= 2:
        print(f"\t{bcolors.FAIL}Multiple template structures found for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id}.{bcolors.ENDC} Skip")
        continue
    template_path = path_solved / str(template_row["path"].item())
    dockq_structure_af = load_PDB(str(structure_path))
    dockq_structure_solved = load_PDB(str(template_path))

    chain_map = {chainA_id: "A", chainB_id:"B"}
    chain_key = chainA_id + chainB_id

    result = run_on_all_native_interfaces(dockq_structure_af, dockq_structure_solved, chain_map=chain_map)[0]
    dataAF.at[i, "DockQ"] = result[chain_key]["DockQ"]
    dataAF.at[i, "iRMSD"] = result[chain_key]["iRMSD"]
    dataAF.at[i, "LRMSD"] = result[chain_key]["LRMSD"]
    dataAF.at[i, "Fnonnat"] = np.float64(result[chain_key]["fnonnat"])

display(dataAF)


,benchmark_set,prediction_name,model_id,model_confidence,chainA_length,chainA_intf_avg_plddt,chainB_length,chainB_intf_avg_plddt,intf_avg_plddt,pDockQ,...,num_align_atoms_domain,num_align_resi_domain,RMSD_all_atom,RMSD_domain,RMSD_backbone_peptide,RMSD_all_atom_peptide,DockQ,iRMSD,LRMSD,Fnonnat
0,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_0,0.722553,499,96.590455,25,87.037000,93.605000,0.278492,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
1,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_1,0.710868,499,96.486842,25,79.521250,91.460000,0.191685,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_2,0.706698,499,97.075000,25,88.953334,94.367778,0.237941,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
3,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_3,0.689345,499,96.403889,25,86.464444,93.090740,0.217522,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
4,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_4,0.679946,499,96.648889,25,84.273749,92.841154,0.213639,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2810,known_extension,TRG_AP2beta_CARGO_1_Mmin_DFL,ranked_0,0.713720,937,88.416363,11,57.284286,76.309444,0.024013,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2811,known_extension,TRG_AP2beta_CARGO_1_Mmin_DFL,ranked_1,0.310416,937,66.480000,11,24.590000,49.724000,0.018259,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2812,known_extension,TRG_AP2beta_CARGO_1_Mmin_DFL,ranked_2,0.209493,937,0.000000,11,0.000000,0.000000,0.000000,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2813,known_extension,TRG_AP2beta_CARGO_1_Mmin_DFL,ranked_3,0.201790,937,0.000000,11,0.000000,0.000000,0.000000,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN


### 5 IPSAE metric
Calculating the new ipSAE metric using the file from the repo ([https://github.com/DunbrackLab/IPSAE/blob/main/ipsae.py](https://github.com/DunbrackLab/IPSAE/blob/main/ipsae.py)). Currently, only AlphaFold 3 is possible as the files from John do not include the confidence.json files anymore. Adds the following column:
- **ipSAE** (all structures): The ipSAE metric for the AlphaFold prediction

In [16]:
def calc_ipsae_metric(row: pd.Series):
    path_cif = path_AF_luck_drive / Path(row["model_path"])
    path_confidences = path_cif.parent / "confidences.json"

    with tempfile.TemporaryDirectory() as tmpdir:
        shutil.copy(path_cif, tmp_path_cif := (Path(tmpdir) / "model.cif"))
        shutil.copy(path_confidences, tmp_path_confidences := (Path(tmpdir) / "confidences.json"))
        subprocess.run(["python", path_ipsae_script, tmp_path_confidences, tmp_path_cif, "10", "10"], env=os.environ.copy())

        path_output = Path(tmpdir) / "model_10_10.txt"

        df_ipsae = pd.read_csv(path_output, header=0, skiprows=[0], sep=" ", skipinitialspace=True)
    return df_ipsae

# For AF2 the json files do not exist anymore
if af_mode == "AF3":
    dataAF["ipSAE"] = np.nan
    for i, row in dataAF.iterrows():
        if row["model_id"] == "ranked_0":
            print(row["prediction_name"], f"({round(100*i/len(dataAF))} %)")
        df_ipsae = calc_ipsae_metric(row)
        dataAF.at[i, "ipSAE"] = np.float64(df_ipsae["ipSAE"][2])
display(dataAF)

,benchmark_set,prediction_name,model_id,model_confidence,chainA_length,chainA_intf_avg_plddt,chainB_length,chainB_intf_avg_plddt,intf_avg_plddt,pDockQ,...,num_align_atoms_domain,num_align_resi_domain,RMSD_all_atom,RMSD_domain,RMSD_backbone_peptide,RMSD_all_atom_peptide,DockQ,iRMSD,LRMSD,Fnonnat
0,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_0,0.722553,499,96.590455,25,87.037000,93.605000,0.278492,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
1,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_1,0.710868,499,96.486842,25,79.521250,91.460000,0.191685,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_2,0.706698,499,97.075000,25,88.953334,94.367778,0.237941,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
3,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_3,0.689345,499,96.403889,25,86.464444,93.090740,0.217522,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
4,known_extension,DEG_APCC_KENBOX_2_M15_M39_D1_D499,ranked_4,0.679946,499,96.648889,25,84.273749,92.841154,0.213639,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2810,known_extension,TRG_AP2beta_CARGO_1_Mmin_DFL,ranked_0,0.713720,937,88.416363,11,57.284286,76.309444,0.024013,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2811,known_extension,TRG_AP2beta_CARGO_1_Mmin_DFL,ranked_1,0.310416,937,66.480000,11,24.590000,49.724000,0.018259,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2812,known_extension,TRG_AP2beta_CARGO_1_Mmin_DFL,ranked_2,0.209493,937,0.000000,11,0.000000,0.000000,0.000000,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2813,known_extension,TRG_AP2beta_CARGO_1_Mmin_DFL,ranked_3,0.201790,937,0.000000,11,0.000000,0.000000,0.000000,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN


### 6 Interaction metrics
Calculate the interaction interface metrics with the newly developed library. Adds the following columns to all structures with an experimentally solved counterpart (None if not):
- **min_distance** [Å]: Distance of the C_alpha atoms (in Angstroms) in the interface if at least one atom pair between the residues is closer than 15 Angstrom. 
- **buried_area** [Å²]: Buried surface area of the interface (in Angstrom squared)
- **disulfide_bonds** [count]: Number of disulfide bonds in the interface
- **salt_bridges** [count]: Number of salt bridges in the interface
- **hbonds** [count]: Number of hydrogen bonds in the interface
- **hydrophobic_interactions** [count]: Number of hydrophobic interactions in the interface

For a detailed explanation see either the library file (measure_PPI.py) or the bachelor thesis (can be found in the group drive)

In [17]:
libpath = Path("../Interface Metrics").resolve()
print(libpath)
sys.path.insert(0, str(libpath))
import measure_PPI

/Users/imb/Interface Metrics


ModuleNotFoundError: No module named 'measure_PPI'

In [8]:
pathObj = []
for i, row in dataAF.iterrows():
    benchmark_set = str(row["benchmark_set"])
    _set = "DDI" if "DDI" in benchmark_set else "DMI"
    prediction_name = str(row["prediction_name"]) if row.notnull()["prediction_name"] else None
    model_id = str(row["model_id"]) if row.notnull()["model_id"] else None

    structure_path = path_AF / _set / benchmark_set / prediction_name / (model_id + ".pdb")
    if not structure_path.exists():
        if row["model_id"] == "ranked_0":
            print(f"\t{bcolors.FAIL}{prediction_name} ({benchmark_set}) does not exist.{bcolors.ENDC} Skip interface metrics")
        continue

    pathObj.append((structure_path.resolve(), prediction_name))
df_intf_metrics = measure_PPI.Run(pathObj=pathObj, num_threads=12)

	PF07724_PF00227_1OFH_C_resi39_resi340.H_resi1_resi172 (known_DDI) does not exist. Skip interface metrics
	PF14978_PF00327_3J7Y_o_resi13_resi101.Z_resi57_resi127 (known_DDI) does not exist. Skip interface metrics
[2025-05-15 19:06:35,957 | measure_PPI | INFO] Started Taskpool of 12 processes for 3170 files
[2025-05-15 19:06:41,001 | measure_PPI | INFO] 2% - ETA 0:03:43 | current speed 14.077 s⁻¹ | average speed 13.878 s⁻¹
[2025-05-15 19:06:46,038 | measure_PPI | INFO] 5% - ETA 0:02:40 | current speed 23.227 s⁻¹ | average speed 18.55 s⁻¹
[2025-05-15 19:06:51,048 | measure_PPI | INFO] 11% - ETA 0:01:53 | current speed 36.922 s⁻¹ | average speed 24.649 s⁻¹
[2025-05-15 19:06:56,112 | measure_PPI | INFO] 17% - ETA 0:01:37 | current speed 34.174 s⁻¹ | average speed 27.042 s⁻¹
[2025-05-15 19:07:01,129 | measure_PPI | INFO] 22% - ETA 0:01:27 | current speed 33.074 s⁻¹ | average speed 28.245 s⁻¹
[2025-05-15 19:07:06,139 | measure_PPI | INFO] 27% - ETA 0:01:20 | current speed 31.148 s⁻¹ | averag

In [9]:
display(df_intf_metrics)

,structure_name,file,hbonds,salt_bridges,buried_area,min_distance,hydrophobic_interactions,disulfide_bonds
2924,D1PF00009_PF01873_2D74.D2PF00026_PF06394_1F34,ranked_0.pdb,1,0,1692.652,4.191,86,0
2925,D1PF00009_PF01873_2D74.D2PF00026_PF06394_1F34,ranked_1.pdb,3,0,1708.317,3.670,96,0
2933,D1PF00009_PF01873_2D74.D2PF00026_PF06394_1F34,ranked_2.pdb,1,1,1907.898,4.643,79,0
2932,D1PF00009_PF01873_2D74.D2PF00026_PF06394_1F34,ranked_3.pdb,5,0,1577.972,3.718,107,0
2934,D1PF00009_PF01873_2D74.D2PF00026_PF06394_1F34,ranked_4.pdb,15,1,2139.539,3.955,101,0
...,...,...,...,...,...,...,...,...
2695,TRG_PTS1_2C0L_NAKL.NAKD,ranked_0.pdb,10,3,839.162,4.782,11,0
2698,TRG_PTS1_2C0L_NAKL.NAKD,ranked_1.pdb,8,3,862.629,4.900,10,0
2696,TRG_PTS1_2C0L_NAKL.NAKD,ranked_2.pdb,12,2,861.115,4.704,9,0
2699,TRG_PTS1_2C0L_NAKL.NAKD,ranked_3.pdb,11,2,854.038,4.943,9,0


In [11]:
dataAF["min_distance"] = None
dataAF["buried_area"] = None
dataAF["disulfide_bonds"] = None
dataAF["salt_bridges"] = None
dataAF["hbonds"] = None
dataAF["hydrophobic_interactions"] = None

for i, row in dataAF.iterrows():
    row_intf = df_intf_metrics[(df_intf_metrics["structure_name"] == row["prediction_name"]) & (df_intf_metrics["file"] == row["model_id"] + ".pdb")]
    if len(row_intf) != 1:
        print(f"\t{bcolors.FAIL}Failed to locate {row["prediction_name"]} {row["model_id"]}{bcolors.ENDC}")
        continue

    dataAF.at[i, "buried_area"] = row_intf["buried_area"].item()
    dataAF.at[i, "min_distance"] = row_intf["min_distance"].item()
    dataAF.at[i, "salt_bridges"] = row_intf["salt_bridges"].item()
    dataAF.at[i, "hbonds"] = row_intf["hbonds"].item()
    dataAF.at[i, "hydrophobic_interactions"] = row_intf["hydrophobic_interactions"].item()
    dataAF.at[i, "disulfide_bonds"] = row_intf["disulfide_bonds"].item()
display(dataAF)

	Failed to locate PF07724_PF00227_1OFH_C_resi39_resi340.H_resi1_resi172 ranked_0
	Failed to locate PF07724_PF00227_1OFH_C_resi39_resi340.H_resi1_resi172 ranked_1
	Failed to locate PF07724_PF00227_1OFH_C_resi39_resi340.H_resi1_resi172 ranked_2
	Failed to locate PF07724_PF00227_1OFH_C_resi39_resi340.H_resi1_resi172 ranked_3
	Failed to locate PF07724_PF00227_1OFH_C_resi39_resi340.H_resi1_resi172 ranked_4
	Failed to locate PF14978_PF00327_3J7Y_o_resi13_resi101.Z_resi57_resi127 ranked_0
	Failed to locate PF14978_PF00327_3J7Y_o_resi13_resi101.Z_resi57_resi127 ranked_1
	Failed to locate PF14978_PF00327_3J7Y_o_resi13_resi101.Z_resi57_resi127 ranked_2
	Failed to locate PF14978_PF00327_3J7Y_o_resi13_resi101.Z_resi57_resi127 ranked_3
	Failed to locate PF14978_PF00327_3J7Y_o_resi13_resi101.Z_resi57_resi127 ranked_4


,project_name,run_id,benchmark_set,prediction_name,model_id,chainA_length,chainB_length,chainA_id,chainB_id,chainA_start,...,DockQ,iRMSD,LRMSD,Fnonnat,buried_area,min_distance,disulfide_bonds,salt_bridges,hbonds,hydrophobic_interactions
0,AlphaFold_benchmark,run37,known_DMI,DEG_APCC_KENBOX_2_4GGD,ranked_0,312,5,A,B,165,...,0.878344,0.603831,1.575394,0.086957,613.651,6.063,0,0,9,0
1,AlphaFold_benchmark,run37,known_DMI,DEG_APCC_KENBOX_2_4GGD,ranked_1,312,5,A,B,165,...,0.880716,0.418230,1.100588,0.050000,580.31,6.083,0,0,9,0
2,AlphaFold_benchmark,run37,known_DMI,DEG_APCC_KENBOX_2_4GGD,ranked_2,312,5,A,B,165,...,0.883186,0.641834,1.776257,0.185185,662.104,6.072,0,0,10,3
3,AlphaFold_benchmark,run37,known_DMI,DEG_APCC_KENBOX_2_4GGD,ranked_3,312,5,A,B,165,...,0.475511,1.686332,5.358800,0.363636,398.498,5.417,0,0,2,0
4,AlphaFold_benchmark,run37,known_DMI,DEG_APCC_KENBOX_2_4GGD,ranked_4,312,5,A,B,165,...,0.223400,2.928606,9.908745,0.888889,323.304,5.092,0,0,2,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3175,AlphaFold_benchmark_DDI,run6,random_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,ranked_0,60,113,B,B,392,...,NaN,NaN,NaN,NaN,1617.382,5.591,0,3,7,56
3176,AlphaFold_benchmark_DDI,run6,random_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,ranked_1,60,113,B,B,392,...,NaN,NaN,NaN,NaN,791.256,6.373,0,0,3,7
3177,AlphaFold_benchmark_DDI,run6,random_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,ranked_2,60,113,B,B,392,...,NaN,NaN,NaN,NaN,882.547,7.906,0,1,2,11
3178,AlphaFold_benchmark_DDI,run6,random_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,ranked_3,60,113,B,B,392,...,NaN,NaN,NaN,NaN,1020.896,4.628,0,3,7,44


### Export table

In [ ]:
# Inspect
enhance_dataframe()
dataAF

,project_name,run_id,benchmark_set,prediction_name,model_id,chainA_length,chainB_length,chainA_id,chainB_id,chainA_start,...,DockQ,iRMSD,LRMSD,Fnonnat,buried_area,min_distance,disulfide_bonds,salt_bridges,hbonds,hydrophobic_interactions
0,AlphaFold_benchmark,run37,known_DMI,DEG_APCC_KENBOX_2_4GGD,ranked_0,312,5,A,B,165,...,0.878344,0.603831,1.575394,0.086957,613.651,6.063,0,0,9,0
1,AlphaFold_benchmark,run37,known_DMI,DEG_APCC_KENBOX_2_4GGD,ranked_1,312,5,A,B,165,...,0.880716,0.418230,1.100588,0.050000,580.310,6.083,0,0,9,0
2,AlphaFold_benchmark,run37,known_DMI,DEG_APCC_KENBOX_2_4GGD,ranked_2,312,5,A,B,165,...,0.883186,0.641834,1.776257,0.185185,662.104,6.072,0,0,10,3
3,AlphaFold_benchmark,run37,known_DMI,DEG_APCC_KENBOX_2_4GGD,ranked_3,312,5,A,B,165,...,0.475511,1.686332,5.358800,0.363636,398.498,5.417,0,0,2,0
4,AlphaFold_benchmark,run37,known_DMI,DEG_APCC_KENBOX_2_4GGD,ranked_4,312,5,A,B,165,...,0.223400,2.928606,9.908745,0.888889,323.304,5.092,0,0,2,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3175,AlphaFold_benchmark_DDI,run6,random_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,ranked_0,60,113,B,B,392,...,NaN,NaN,NaN,NaN,1617.382,5.591,0,3,7,56
3176,AlphaFold_benchmark_DDI,run6,random_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,ranked_1,60,113,B,B,392,...,NaN,NaN,NaN,NaN,791.256,6.373,0,0,3,7
3177,AlphaFold_benchmark_DDI,run6,random_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,ranked_2,60,113,B,B,392,...,NaN,NaN,NaN,NaN,882.547,7.906,0,1,2,11
3178,AlphaFold_benchmark_DDI,run6,random_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,ranked_3,60,113,B,B,392,...,NaN,NaN,NaN,NaN,1020.896,4.628,0,3,7,44


In [ ]:
# Save
enhance_dataframe()
dataAF.to_csv(path_AF / (path_AF.name + "_metrics.tsv"), sep="\t", index=None)
dataAF.to_excel(path_AF/ (path_AF.name + "_metrics.xlsx"), sheet_name=f"{af_mode} metrics", index=None)